# Modeling Neural Population with Gaussian Mixture Models

The present work explores how one can model learning in toy neural populations as an insatnce of online Gaussian Mixture Model learning (GMM). First we descirbe the K-means algorithm, or whats the same, the Hard Expectation-Maximization (EM) algorithm with spherical vrainces. Then we show how this algorithm can be seen as an istance of a Winner Takes it All (WTA) network with Hebbian-Learning. 

We then briefly describe why the EM algorithm is convenient to model the system at hand and continue to relax the simplifications made in the Hard EM algorithm to give way tho the soft EM algorithm. Finally we touch on the implications that lifting these simplifications has on the analogue nerual model. 

## The Hard EM Algorithm
### GMMs and the EM algorithm

Hidden variable models seek to explain observed variables in terms of latent ones. The EM algorithm allowes us to do maximum likelihood estimation (MLE) or maximum a posteriori (MAP) estimation with models that have hidden random variables (RVs). 

The EM algorithm finds one of it's use cases is clasification problems, where one seeks to label unlabled data points. In this context the hidden variables are the cluster identities and the observed variables are the data points. Furthermore if we assume that the distribution of the data given the identity of a cluster is Gaussian, then this gives way to GMMs. 

Hence, we can write the conditional porbability of a specific data point as a linear combination of a set of Gaussians. 

$$
P(x \mid \mu, \Sigma, w) = \sum_{i=1}^{k} w_i N(x; \mu_i, \Sigma_i)
$$

Here $x$ is our data point, and $\mu_i$ and $\Sigma_i$ represent the mean and covariance matrix of cluster $i$.

### The Hard EM 
The EM algorithm iterates until convergence. One iteration of the EM starts by performming an Expectations step (E-step) followed by a Maximization step (M-step).

The EM iteration alternates between performing an expectation (E) step, which creates a function for the expectation of the log-likelihood evaluated using the current estimate for the parameters, and a maximization (M) step, which computes parameters maximizing the expected log-likelihood found on the E step. These parameter-estimates are then used to determine the distribution of the latent variables in the next E step.

TODO: Describe the Hard EM algorithm (why is it called Hard Santi?)

### The Hard EM and K-means
Moreover if we fix the weights to be uiform and the covariances to be the same and spherical the EM algorithm becomes one and the same with the vanilla K-means algorithm. 

$$
w_1 = w_2 = ... = w_k = \frac{1}{k} \\
\Sigma_1 = \Sigma_2 = ... = \Sigma_k = \sigma^2 \; I
$$